# Practice Scenario: Logistic Regression  

*You're working as a data analyst for an e-commerce company. Your goal is to help the marketing team identify which website visitors are most likely to make a purchase, based on their behavior and profile.*

---

You will need a dataset with features like: Age, Gender, time-on-site, pages-viewed, traffic-source (google, facebook, direct), location, device-type. With a target: PURCHASED (1 or 0)

In [33]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve, confusion_matrix
from statsmodels.api import OLS, add_constant
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', None)

### Import data 

In [34]:
data = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
display(data.head(3))
display(data.info())

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


None

In [35]:
data['MultipleLines'].value_counts()

MultipleLines
No                  3390
Yes                 2971
No phone service     682
Name: count, dtype: int64

### Feature Engineering / Cleaning

Features must be either binary or continuous. They can be categorical if they are converted to dummy variables.

In [36]:
data['MultipleLines'].value_counts()

MultipleLines
No                  3390
Yes                 2971
No phone service     682
Name: count, dtype: int64

In [37]:
data["is_male"] = data['gender'].map({"Female": 0, "Male": 1}) #Convert to binary
data['Partner'] = data['Partner'].map({'Yes':1, 'No':0}) #Convert to 0/1 for partner
data['Dependents'] = data['Dependents'].map({'Yes':1, 'No':0}) #Convert to 0/1 for dependents
data['PhoneService'] = data['PhoneService'].map({'Yes':1, 'No':0})
data['MultipleLines'] = data['MultipleLines'].map({'Yes':1, 'No':0, 'No phone service':0})
data['OnlineSecurity'] = data['OnlineSecurity'].map({'Yes':1, 'No':0})
data['OnlineBackup'] = data['OnlineBackup'].map({'Yes':1, 'No':0})
data['DeviceProtection'] = data['DeviceProtection'].map({'Yes':1, 'No':0})
data['TechSupport'] = data['TechSupport'].map({'Yes':1, 'No':0})
data['StreamingTV'] = data['StreamingTV'].map({'Yes':1, 'No':0})
data['StreamingMovies'] = data['StreamingMovies'].map({'Yes':1, 'No':0})
data['PaperlessBilling'] = data['PaperlessBilling'].map({'Yes':1, 'No':0})
data['Churn'] = data['Churn'].map({'Yes':1, 'No':0})

df_cleaned = pd.get_dummies(data, columns=['InternetService', 'Contract', 'PaymentMethod'], drop_first=True, dtype=int)
display(df_cleaned.head(3))
display(df_cleaned.info())

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,is_male,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,7590-VHVEG,Female,0,1,0,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1,29.85,29.85,0,0,0,0,0,0,0,1,0
1,5575-GNVDE,Male,0,0,0,34,1,0,1.0,0.0,1.0,0.0,0.0,0.0,0,56.95,1889.5,0,1,0,0,1,0,0,0,1
2,3668-QPYBK,Male,0,0,0,2,1,0,1.0,1.0,0.0,0.0,0.0,0.0,1,53.85,108.15,1,1,0,0,0,0,0,0,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 26 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   customerID                             7043 non-null   object 
 1   gender                                 7043 non-null   object 
 2   SeniorCitizen                          7043 non-null   int64  
 3   Partner                                7043 non-null   int64  
 4   Dependents                             7043 non-null   int64  
 5   tenure                                 7043 non-null   int64  
 6   PhoneService                           7043 non-null   int64  
 7   MultipleLines                          7043 non-null   int64  
 8   OnlineSecurity                         5517 non-null   float64
 9   OnlineBackup                           5517 non-null   float64
 10  DeviceProtection                       5517 non-null   float64
 11  Tech

None

In [80]:
new_df = df_cleaned.copy()
df = new_df.drop(columns=['gender', 'customerID'])
df

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,is_male,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,1,0,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1,29.85,29.85,0,0,0,0,0,0,0,1,0
1,0,0,0,34,1,0,1.0,0.0,1.0,0.0,0.0,0.0,0,56.95,1889.5,0,1,0,0,1,0,0,0,1
2,0,0,0,2,1,0,1.0,1.0,0.0,0.0,0.0,0.0,1,53.85,108.15,1,1,0,0,0,0,0,0,1
3,0,0,0,45,0,0,1.0,0.0,1.0,1.0,0.0,0.0,0,42.30,1840.75,0,1,0,0,1,0,0,0,0
4,0,0,0,2,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,70.70,151.65,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,1,1,24,1,1,1.0,0.0,1.0,1.0,1.0,1.0,1,84.80,1990.5,0,1,0,0,1,0,0,0,1
7039,0,1,1,72,1,1,0.0,1.0,1.0,0.0,1.0,1.0,1,103.20,7362.9,0,0,1,0,1,0,1,0,0
7040,0,1,1,11,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1,29.60,346.45,0,0,0,0,0,0,0,1,0
7041,1,1,0,4,1,1,0.0,0.0,0.0,0.0,0.0,0.0,1,74.40,306.6,1,1,1,0,0,0,0,0,1


In [83]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())
df.dtypes

SeniorCitizen                              int64
Partner                                    int64
Dependents                                 int64
tenure                                     int64
PhoneService                               int64
MultipleLines                              int64
OnlineSecurity                           float64
OnlineBackup                             float64
DeviceProtection                         float64
TechSupport                              float64
StreamingTV                              float64
StreamingMovies                          float64
PaperlessBilling                           int64
MonthlyCharges                           float64
TotalCharges                             float64
Churn                                      int64
is_male                                    int64
InternetService_Fiber optic                int32
InternetService_No                         int32
Contract_One year                          int32
Contract_Two year   

In [84]:
display(df.isna().sum())
df.count()

SeniorCitizen                               0
Partner                                     0
Dependents                                  0
tenure                                      0
PhoneService                                0
MultipleLines                               0
OnlineSecurity                           1526
OnlineBackup                             1526
DeviceProtection                         1526
TechSupport                              1526
StreamingTV                              1526
StreamingMovies                          1526
PaperlessBilling                            0
MonthlyCharges                              0
TotalCharges                                0
Churn                                       0
is_male                                     0
InternetService_Fiber optic                 0
InternetService_No                          0
Contract_One year                           0
Contract_Two year                           0
PaymentMethod_Credit card (automat

SeniorCitizen                            7043
Partner                                  7043
Dependents                               7043
tenure                                   7043
PhoneService                             7043
MultipleLines                            7043
OnlineSecurity                           5517
OnlineBackup                             5517
DeviceProtection                         5517
TechSupport                              5517
StreamingTV                              5517
StreamingMovies                          5517
PaperlessBilling                         7043
MonthlyCharges                           7043
TotalCharges                             7043
Churn                                    7043
is_male                                  7043
InternetService_Fiber optic              7043
InternetService_No                       7043
Contract_One year                        7043
Contract_Two year                        7043
PaymentMethod_Credit card (automat

In [85]:
df['PhoneService'][df['OnlineSecurity'].isna()].sum()

1526

Here I am just checking the total number of NAN's per column. Now we can see that in the areas of Online Security, Online Backup, Device protection, TechSupport, StreamingTV, and Streaming Movies, they all have the same number of NANs. Alltogether if we do remove the NANs it would be around 20% of the entire dataset. But then if you look at the 'Phone service' column which states whether the customer has internet or not, it shows that the ones that don't have these services also don't have internet which would make sense so rather than remove, I would replace them all with 0.

In [86]:
df = df.fillna(0)
display(df.isna().sum())
df.head(3)

SeniorCitizen                            0
Partner                                  0
Dependents                               0
tenure                                   0
PhoneService                             0
MultipleLines                            0
OnlineSecurity                           0
OnlineBackup                             0
DeviceProtection                         0
TechSupport                              0
StreamingTV                              0
StreamingMovies                          0
PaperlessBilling                         0
MonthlyCharges                           0
TotalCharges                             0
Churn                                    0
is_male                                  0
InternetService_Fiber optic              0
InternetService_No                       0
Contract_One year                        0
Contract_Two year                        0
PaymentMethod_Credit card (automatic)    0
PaymentMethod_Electronic check           0
PaymentMeth

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,is_male,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,1,0,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1,29.85,29.85,0,0,0,0,0,0,0,1,0
1,0,0,0,34,1,0,1.0,0.0,1.0,0.0,0.0,0.0,0,56.95,1889.50,0,1,0,0,1,0,0,0,1
2,0,0,0,2,1,0,1.0,1.0,0.0,0.0,0.0,0.0,1,53.85,108.15,1,1,0,0,0,0,0,0,1


### Exploratory Data Analysis

In [88]:
df.corr()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,is_male,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
SeniorCitizen,1.000000,0.016479,-0.211185,0.016567,0.008576,0.142948,-0.038653,0.066572,0.059428,-0.060625,0.105378,0.120176,0.156530,0.220173,0.102652,0.150889,-0.001874,0.255338,-0.182742,-0.046262,-0.117000,-0.024135,0.171718,-0.153477
Partner,0.016479,1.000000,0.452676,0.379697,0.017706,0.142057,0.143106,0.141498,0.153786,0.119999,0.124666,0.117412,-0.014877,0.096848,0.318364,-0.150448,-0.001808,0.000304,0.000615,0.082783,0.248091,0.082029,-0.083852,-0.095125
Dependents,-0.211185,0.452676,1.000000,0.159712,-0.001762,-0.024526,0.080972,0.023671,0.013963,0.063268,-0.016558,-0.039741,-0.111377,-0.113890,0.063593,-0.164221,0.010517,-0.165818,0.139812,0.068368,0.204613,0.060267,-0.150642,0.059071
tenure,0.016567,0.379697,0.159712,1.000000,0.008448,0.331941,0.327203,0.360277,0.360653,0.324221,0.279756,0.286111,0.006152,0.247900,0.825464,-0.352229,0.005106,0.019720,-0.039062,0.202570,0.558533,0.233006,-0.208363,-0.233852
PhoneService,0.008576,0.017706,-0.001762,0.008448,1.000000,0.279690,-0.092893,-0.052312,-0.071227,-0.096340,-0.022574,-0.032959,0.016505,0.247398,0.113013,0.011942,-0.006488,0.289999,0.172209,-0.002791,0.003519,-0.007721,0.003062,-0.003319
MultipleLines,0.142948,0.142057,-0.024526,0.331941,0.279690,1.000000,0.098108,0.202237,0.201137,0.100571,0.257152,0.258751,0.163530,0.490434,0.468705,0.040102,-0.008414,0.366083,-0.210564,-0.003794,0.106253,0.060048,0.083618,-0.227206
OnlineSecurity,-0.038653,0.143106,0.080972,0.327203,-0.092893,0.098108,1.000000,0.283832,0.275438,0.354931,0.176207,0.187398,-0.003636,0.296594,0.412091,-0.171226,-0.017021,-0.030696,-0.333403,0.100162,0.191773,0.115721,-0.112338,-0.080798
OnlineBackup,0.066572,0.141498,0.023671,0.360277,-0.052312,0.202237,0.283832,1.000000,0.303546,0.294233,0.282106,0.274501,0.126735,0.441780,0.509607,-0.082255,-0.013773,0.165651,-0.381593,0.083722,0.111400,0.090785,-0.000408,-0.174164
DeviceProtection,0.059428,0.153786,0.013963,0.360653,-0.071227,0.201137,0.275438,0.303546,1.000000,0.333313,0.390874,0.402111,0.103797,0.482692,0.522374,-0.066160,-0.002105,0.176049,-0.380754,0.102495,0.165096,0.111554,-0.003351,-0.187373
TechSupport,-0.060625,0.119999,0.063268,0.324221,-0.096340,0.100571,0.354931,0.294233,0.333313,1.000000,0.278070,0.279358,0.037880,0.338304,0.432329,-0.164674,-0.009212,-0.020492,-0.336298,0.095775,0.240824,0.117272,-0.114839,-0.085509
